# Assignment: Segmenting & Clustering Neigborhoods in Toronto - (Part3: Clustering)

To complete the last stage of our assignment we would be clustering the the neighborhoods in Toronto city. We would be using k-means technique to implemnt clustering. But before that we would need to repeat some of the steps completed in Part 2 of our assignment.

To start with we would intall 2 packages which would be neccessary to us in this part of the assignment, Folium and Geopy. The former will be used for plotting maps and the latter would be used for Geocodes and getting the coordinates.

In [2]:
!pip install folium
print('folium successfully downloaded!')

folium successfully downloaded!


In [23]:
!pip install geopy
print('Geopy successfully downloaded!')

Geopy successfully downloaded!


#### Import the necceessary Libraries

In [1]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs
print('All libraries imported!')

All libraries imported!


#### Reload the DataFrame from Part 2

In [2]:
toronto_pin = pd.read_csv('toronto_complete.csv')
toronto_pin.head(10)

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M9A,Etobicoke,Islington Avenue,43.806686,-79.194353
1,1,M4Y,Downtown Toronto,Church and Wellesley,43.784535,-79.160497
2,2,M2R,North York,Willowdale West,43.763573,-79.188711
3,3,M8Z,Etobicoke,"Mimico NW, The Queensway West",43.770992,-79.216917
4,4,M9C,Etobicoke,Markland Wood,43.773136,-79.239476
5,5,M4T,Central Toronto,Moore Park,43.744734,-79.239476
6,6,M3A,North York,Parkwoods,43.727929,-79.262029
7,7,M1K,Scarborough,"Ionview, Kennedy Park",43.711112,-79.284577
8,8,M6M,York,"Keelesdale, Mount Dennis, Silverthorn",43.716316,-79.239476
9,9,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill",43.692657,-79.264848


#### Find the coordinates of Toronto

In [3]:
city = 'Toronto'
geolocator = Nominatim()
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\arjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto

In [4]:
#use folium to create a map of Toronto city
toronto_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_pin['Latitude'], toronto_pin['Longitude'], toronto_pin['Borough'], toronto_pin['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    

toronto_map

#### Create a DataFrame with neighborhoods in North York

In [5]:
# using the Foursquare account credentials
CLIENT_ID = 'CPASOOY1KMCZKKSQQUBE0GKMRUSP5JZUQK50T3Z03V4IS2IO' 
CLIENT_SECRET = 'UTUOMX4ASKLR0XBRG2XEDDSQSKU1A1RRVKGQZINMGFJPHO24'
VERSION = '20180605' 

In [6]:
north_york_data = toronto_pin[toronto_pin['Borough'] == 'North York'].reset_index(drop=True)
north_york_data.head(10)

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,2,M2R,North York,Willowdale West,43.763573,-79.188711
1,6,M3A,North York,Parkwoods,43.727929,-79.262029
2,15,M2K,North York,Bayview Village,43.799525,-79.318389
3,20,M5M,North York,Bedford Park,43.757490,-79.374714
4,26,M3L,North York,Downsview West,43.745906,-79.352188
5,32,M2H,North York,Hillcrest Village,43.728496,-79.495697
6,36,M2J,North York,Henry Farm,43.695344,-79.318389
7,39,M9L,North York,Humber Summit,43.705369,-79.349372
8,41,M6L,North York,Downsview,43.679557,-79.352188
9,43,M3C,North York,Flemingdon Park,43.659526,-79.340923


#### Create a map of the North York neighborhood

In [7]:
borough_ny = 'North York,Toronto'
latitude_ny = 43.7615
longitude_ny = -79.4111
print('The coordinate of North York are {}, {}.'.format(latitude_ny, longitude_ny))

The coordinate of North York are 43.7615, -79.4111.


In [8]:
ny_map = folium.Map(location=[latitude_ny, longitude_ny], zoom_start=12)

# add markers to map
for lat, lng, label in zip(north_york_data['Latitude'], north_york_data['Longitude'], north_york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(ny_map)  
    
ny_map

#### Within North York, get the top 100 venues in neighborhood 'Willowdale West'

In [10]:
neighborhood_latitude = north_york_data.loc[0, 'Latitude'] 
neighborhood_longitude = north_york_data.loc[0, 'Longitude'] 

neighborhood_name = north_york_data.loc[0, 'Neighbourhood'] 

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Willowdale West" are 43.7635726, -79.1887115.


In [12]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ny, longitude_ny, VERSION, radius, LIMIT)

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d0077283183940025bb399e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Willowdale',
  'headerFullLocation': 'Willowdale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 41,
  'suggestedBounds': {'ne': {'lat': 43.7660000045, 'lng': -79.4048808833373},
   'sw': {'lat': 43.756999995499996, 'lng': -79.41731911666271}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '55665a0b498ec5589987b1f7',
       'name': 'Kinka Izakaya',
       'location': {'address': '4775 Yonge Street, Unit #114',
        'crossStreet': 'Sheppard Ave.',
        'lat': 43.76016102214242,
        'lng': -79.40982686116466,
        'labeledLatLngs':

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
# library to handle JSON files
# tranform JSON file into a pandas dataframe
import json 
from pandas.io.json import json_normalize 

# flatten JSON
venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) 

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(10)

,name,categories,lat,lng
0,Kinka Izakaya,Japanese Restaurant,43.760161,-79.409827
1,Sushi Moto Sake & Wine Bar,Sushi Restaurant,43.763902,-79.411559
2,EatBKK,Thai Restaurant,43.759320,-79.410454
3,Bake Code 烘焙密碼,Bakery,43.763969,-79.411641
4,MYMY Chicken,Fried Chicken Joint,43.764658,-79.411096
5,The Fry,Fried Chicken Joint,43.762993,-79.411442
6,Whole Foods Market,Grocery Store,43.760662,-79.411044
7,Hero Certified Burgers,Burger Joint,43.758885,-79.410249
8,Nolbu,Korean Restaurant,43.763029,-79.411937
9,Union Social Eatery,Restaurant,43.763797,-79.411358


In [17]:
print('{} Total venues returned by Foursquare.'.format(nearby_venues.shape[0]))

41 Total venues returned by Foursquare.


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

#### Listing all the venues for each neighborhood in North York

In [19]:
north_york_venues = getNearbyVenues(names=north_york_data['Neighbourhood'],
                                   latitudes=north_york_data['Latitude'],
                                   longitudes=north_york_data['Longitude']
                                   )

Willowdale West
Parkwoods
Bayview Village
Bedford Park
Downsview West
Hillcrest Village
Henry Farm
Humber Summit
Downsview
Flemingdon Park
Victoria Village
Bathurst Manor, Wilson Heights
Emery, Humberlea
Lawrence Heights, Lawrence Manor
CFB Toronto
Northwood Park, York University
Newtonbrook, Willowdale
York Mills


In [20]:
north_york_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Willowdale West,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
1,Willowdale West,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,Willowdale West,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
3,Willowdale West,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,Willowdale West,43.763573,-79.188711,chatr Mobile,43.765917,-79.191672,Tech Startup
5,Willowdale West,43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
6,Willowdale West,43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
7,Willowdale West,43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
8,Willowdale West,43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
9,Parkwoods,43.727929,-79.262029,Giant Tiger,43.727447,-79.266240,Department Store


In [21]:
north_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights",9,9,9,9,9,9
Bayview Village,12,12,12,12,12,12
Bedford Park,1,1,1,1,1,1
CFB Toronto,2,2,2,2,2,2
Downsview,42,42,42,42,42,42
Downsview West,5,5,5,5,5,5
"Emery, Humberlea",14,14,14,14,14,14
Flemingdon Park,38,38,38,38,38,38
Henry Farm,8,8,8,8,8,8


In [23]:
print('Number of unique categories is: {}'.format(len(north_york_venues['Venue Category'].unique())))

Number of unique categories is: 149


In [24]:
# one hot encoding
ny_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Yoga Studio,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head()

,Neighborhood,Yoga Studio,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,...,Theater,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Women's Store
0,"Bathurst Manor, Wilson Heights",0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bayview Village,0.00000,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.083333,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bedford Park,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CFB Toronto,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Downsview,0.02381,0.023810,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.02381,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Get top 10 venues per Neighborhood

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights",Grocery Store,Playground,Food & Drink Shop,Sandwich Place,Clothing Store,Breakfast Spot,Hotel,Gym,Park,Convenience Store
1,Bayview Village,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Thrift / Vintage Store,Coffee Shop,Pizza Place,Sandwich Place,Breakfast Spot,Grocery Store,American Restaurant
2,Bedford Park,Cafeteria,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store,Diner,Dessert Shop,Women's Store
3,CFB Toronto,Garden,Ice Cream Shop,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store,Diner,Dessert Shop,Department Store
4,Downsview,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Liquor Store,Spa,Brewery,Bubble Tea Shop
5,Downsview West,Japanese Restaurant,Café,Gym / Fitness Center,Baseball Field,Caribbean Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store,Diner
6,"Emery, Humberlea",Coffee Shop,Pub,Bagel Shop,Sports Bar,Supermarket,Sushi Restaurant,Light Rail Station,Fried Chicken Joint,Liquor Store,Pizza Place
7,Flemingdon Park,Café,Coffee Shop,American Restaurant,Gastropub,Italian Restaurant,Bakery,Yoga Studio,Convenience Store,Sandwich Place,Cheese Shop
8,Henry Farm,Park,Bus Stop,Pharmacy,Skating Rink,Cosmetics Shop,Beer Store,Athletics & Sports,Curling Ice,Eastern European Restaurant,Donut Shop
9,Hillcrest Village,Home Service,Food Truck,Korean Restaurant,Baseball Field,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store


#### Implement k-means clustering and cluster the neighborhoods into 5 clusters

In [49]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

ny_data = north_york_data.drop(17)
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_=kmeans.labels_[0:17]
print(kmeans.labels_)
print(len(kmeans.labels_))
print(ny_data.shape)


[0 0 3 2 0 0 0 0 0 4 0 0 0 0 0 1 0]
17
(17, 6)


#### Include kmeans labels into the original North York dataframe

In [50]:
ny_merged = ny_data

# add clustering labels
ny_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

ny_merged


,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M2R,North York,Willowdale West,43.763573,-79.188711,0,Electronics Store,Intersection,Breakfast Spot,Medical Center,Mexican Restaurant,Rental Car Location,Tech Startup,Pizza Place,Spa,Creperie
1,6,M3A,North York,Parkwoods,43.727929,-79.262029,0,Coffee Shop,Discount Store,Department Store,Playground,Gourmet Shop,Electronics Store,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop
2,15,M2K,North York,Bayview Village,43.799525,-79.318389,3,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Thrift / Vintage Store,Coffee Shop,Pizza Place,Sandwich Place,Breakfast Spot,Grocery Store,American Restaurant
3,20,M5M,North York,Bedford Park,43.757490,-79.374714,2,Cafeteria,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store,Diner,Dessert Shop,Women's Store
4,26,M3L,North York,Downsview West,43.745906,-79.352188,0,Japanese Restaurant,Café,Gym / Fitness Center,Baseball Field,Caribbean Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store,Diner
5,32,M2H,North York,Hillcrest Village,43.728496,-79.495697,0,Home Service,Food Truck,Korean Restaurant,Baseball Field,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store
6,36,M2J,North York,Henry Farm,43.695344,-79.318389,0,Park,Bus Stop,Pharmacy,Skating Rink,Cosmetics Shop,Beer Store,Athletics & Sports,Curling Ice,Eastern European Restaurant,Donut Shop
7,39,M9L,North York,Humber Summit,43.705369,-79.349372,0,Indian Restaurant,Yoga Studio,Pizza Place,Liquor Store,Discount Store,Supermarket,Intersection,Sandwich Place,Bank,Housing Development
8,41,M6L,North York,Downsview,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Liquor Store,Spa,Brewery,Bubble Tea Shop
9,43,M3C,North York,Flemingdon Park,43.659526,-79.340923,4,Café,Coffee Shop,American Restaurant,Gastropub,Italian Restaurant,Bakery,Yoga Studio,Convenience Store,Sandwich Place,Cheese Shop


#### Visualize the clusters in the map

In [51]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_ny, longitude_ny], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighbourhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Explore each of the 5 clusters

In [52]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5,ny_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2R,-79.188711,0,Electronics Store,Intersection,Breakfast Spot,Medical Center,Mexican Restaurant,Rental Car Location,Tech Startup,Pizza Place,Spa,Creperie
1,M3A,-79.262029,0,Coffee Shop,Discount Store,Department Store,Playground,Gourmet Shop,Electronics Store,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop
4,M3L,-79.352188,0,Japanese Restaurant,Café,Gym / Fitness Center,Baseball Field,Caribbean Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store,Diner
5,M2H,-79.495697,0,Home Service,Food Truck,Korean Restaurant,Baseball Field,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store
6,M2J,-79.318389,0,Park,Bus Stop,Pharmacy,Skating Rink,Cosmetics Shop,Beer Store,Athletics & Sports,Curling Ice,Eastern European Restaurant,Donut Shop
7,M9L,-79.349372,0,Indian Restaurant,Yoga Studio,Pizza Place,Liquor Store,Discount Store,Supermarket,Intersection,Sandwich Place,Bank,Housing Development
8,M6L,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Yoga Studio,Liquor Store,Spa,Brewery,Bubble Tea Shop
10,M4A,-79.388790,0,Park,Swim School,Bus Line,Farmers Market,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store,Diner
11,M3H,-79.390197,0,Grocery Store,Playground,Food & Drink Shop,Sandwich Place,Clothing Store,Breakfast Spot,Hotel,Gym,Park,Convenience Store
12,M9M,-79.400049,0,Coffee Shop,Pub,Bagel Shop,Sports Bar,Supermarket,Sushi Restaurant,Light Rail Station,Fried Chicken Joint,Liquor Store,Pizza Place


In [53]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,M3J,-79.374846,1,Coffee Shop,Café,Restaurant,Hotel,Cocktail Bar,Seafood Restaurant,Italian Restaurant,Beer Bar,Park,Farmers Market


In [54]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M5M,-79.374714,2,Cafeteria,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Discount Store,Diner,Dessert Shop,Women's Store


In [55]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M2K,-79.318389,3,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Thrift / Vintage Store,Coffee Shop,Pizza Place,Sandwich Place,Breakfast Spot,Grocery Store,American Restaurant


In [56]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M3C,-79.340923,4,Café,Coffee Shop,American Restaurant,Gastropub,Italian Restaurant,Bakery,Yoga Studio,Convenience Store,Sandwich Place,Cheese Shop
